In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 27.2 MB/s eta 0:00:00


In [ ]:
#for college email and 8500
import openai
import os
os.environ["OPENAI_API_KEY"] ='your api key'
openai.api_key ='your api key'

In [36]:
import json
with open ('/content/drive/MyDrive/MTP Project/t5_data/test_data_25_each.json') as f:
  test=json.load(f)
test

[{'sentence': 'The total length of this LPG pipeline is 1415 km .',
  'entity': 'LPG pipeline',
  'measuring attribute': 'total length',
  'quantity': '1415 km'},
 {'sentence': 'Defensive end Rulon Jones led the line with 7 sacks .',
  'entity': 'Rulon Jones',
  'measuring attribute': 'number of sacks',
  'quantity': '7 sacks'},
 {'sentence': "'' The diameter of an Oreo cookie is 1.75 inches .",
  'entity': 'Oreo cookie',
  'measuring attribute': 'diameter',
  'quantity': '1.75 inches'},
 {'sentence': 'This presence in Russia lasted 3 years .',
  'entity': 'unknown',
  'measuring attribute': 'presence duration',
  'quantity': '3 years'},
 {'sentence': 'Hyundai owns 33.88 % of Kia .',
  'entity': 'Hyundai',
  'measuring attribute': 'Kia share',
  'quantity': '33.88 %'},
 {'sentence': 'The Criollos entered the league \'s " Super 6 " semifinals .',
  'entity': 'Criollos',
  'measuring attribute': 'unknown',
  'quantity': '6 "'},
 {'sentence': "Vermont 's share is about 4.5 percent .",
  '

## Input - Sentence, quant
##Output - ent , attr ----- 200 samples

In [ ]:
#load the annotated dataset
import json
with open('/content/drive/MyDrive/MTP Project/t5_data/annotated_200.json', 'r') as f:
    samples = json.load(f)
sent={}
for i in samples:
  sent[i['sentence']]=[i['entity'],i['quantity'],i['attribute']]
len(sent)

In [30]:
#prepare the dataset for joint entity and attribute extraction
output=[]
for i,v in sent.items():
  p=f"Given the sentence '''{i}''' and the quantity '''{v[1]}''', extract the entity and attribute."
  #prompt= f"Please extract the entity and attribute related to the given quantity in the sentence. Sentence:{i} Quantity:{v[1]}"
  comp=f"Entity: {v[0]} ; Attribute: {v[2]}"
  line = {'prompt': p, 'completion': comp}

  output.append(line)


In [31]:
output[0]

{'prompt': "Given the sentence '''Without the spire , Burj Khalifa would be 585 meters tall .''' and the quantity '''585 meters''', extract the entity and attribute.",
 'completion': 'Entity: Burj Khalifa ; Attribute: height'}

In [ ]:
import json
# Open the file 'out_ent_attr.jsonl' in write mode
with open('/content/drive/MyDrive/MTP Project/gpt_data/out_ent_attr.jsonl', 'w') as outfile:
    # Iterate over the 'output' list
    for i in output:
        # Write each object from the 'output' list as a JSON string to the file
        json.dump(i, outfile)
        # Write a new line character to separate each object
        outfile.write('\n')


In [ ]:
#prepare the data
!openai tools fine_tunes.prepare_data -f '/content/drive/MyDrive/MTP Project/gpt_data/out_ent_attr.jsonl'

Analyzing...

- Your file contains 200 prompt-completion pairs
- All prompts end with suffix `''', extract the entity and attribute.`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- All prompts start with prefix `Given the sentence '''`. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- All completions start with prefix `Entity: `. Most of the time you should only add the output data into the completion, without any prefix
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitesp

In [ ]:
#fine tune the davinci model
!openai api fine_tunes.create -t "/content/drive/MyDrive/MTP Project/gpt_data/out_ent_attr_prepared.jsonl" -m davinci

Upload progress: 100% 59.2k/59.2k [00:00<00:00, 50.2Mit/s]
Uploaded file from /content/drive/MyDrive/MTP Project/gpt_data/out_ent_attr_prepared.jsonl: file-kAjZKcc4sTxhsWVIcYzYFfEG
Created fine-tune: ft-zdvhTtcusnAsaxCrPKfFP4EV
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-24 12:34:52] Created fine-tune: ft-zdvhTtcusnAsaxCrPKfFP4EV
[2023-04-24 12:35:04] Fine-tune costs $1.52
[2023-04-24 12:35:04] Fine-tune enqueued. Queue number: 0
[2023-04-24 12:35:06] Fine-tune started



In [ ]:
!openai api fine_tunes.follow -i ft-zdvhTtcusnAsaxCrPKfFP4EV

[2023-04-24 12:34:52] Created fine-tune: ft-zdvhTtcusnAsaxCrPKfFP4EV
[2023-04-24 12:35:04] Fine-tune costs $1.52
[2023-04-24 12:35:04] Fine-tune enqueued. Queue number: 0
[2023-04-24 12:35:06] Fine-tune started
[2023-04-24 12:37:37] Completed epoch 1/4
[2023-04-24 12:38:38] Completed epoch 2/4
[2023-04-24 12:39:39] Completed epoch 3/4
[2023-04-24 12:40:40] Completed epoch 4/4
[2023-04-24 12:41:13] Uploaded model: davinci:ft-personal-2023-04-24-12-41-13
[2023-04-24 12:41:15] Uploaded result file: file-TMjgWyUzwMvCZxt1IumzLfhD
[2023-04-24 12:41:15] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-04-24-12-41-13 -p <YOUR_PROMPT>


In [ ]:
test[0]

{'entity': 'this LPG pipeline',
 'quantity': '1415 km',
 'attribute': 'total length',
 'sentence': 'The total length of this LPG pipeline is 1415 km .'}

In [ ]:
# Set the model_engine variable to the specified engine for fine-tuned model
model_engine = 'davinci:ft-personal-2023-04-24-12-41-13'

# Initialize variables
m = 0
out = {}
import time

# Iterate over the test list
for se in test:
    # Create a prompt string using the values from the current element in the test list
    p = f"Given the sentence '''{se['sentence']}''' and the quantity '''{se['quantity']}''', extract the entity and attribute."

    # Generate completions using the fine-tuned model
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=p,
        temperature=0.7,
        max_tokens=30,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        n=1
    )

    # Get the first completion from the generated completions
    message = completions.choices[0].text

    # Print the original sentence and the generated completion
    print(se['sentence'])
    print("*****************************Result**************************")
    print(message.split('\n')[0])
    print('\n\n')

    # Store the generated completion in the out dictionary using the original sentence as the key
    out[se['sentence']] = message.split('\n')[0]

    m = m + 1
    # Delay execution every 20 iterations to avoid rate limits
    if m % 20 == 0:
        time.sleep(65)


In [ ]:
#seperate entity and attribute in the predicted output
output=[]
# Define regular expressions to match the entity and attribute values
entity_regex = r"Entity:\s*([\w\s]+)"
attribute_regex = r"Attribute:\s*([\w\s]+)"

for i,v in out.items():
  print(i)
  print(f'*******************   RESULT   *******************   ')
  input=v
# Use regular expressions to extract the entity and attribute values from the input string
  entity_match = re.search(entity_regex, input)
  attribute_match = re.search(attribute_regex, input)
# Extract the entity and attribute values from the regular expression matches
  entity_value = entity_match.group(1).strip()
  attribute_value = attribute_match.group(1).strip()
  print(f'Entity----{entity_value}')
  print(f"Attribute ----{attribute_value}")

  print('\n\n')
  d={}
  d['sentence']=i
  d['entity']=entity_value
  d['attribute']=attribute_value
  output.append(d)


In [ ]:
with open('/content/drive/MyDrive/MTP Project/gpt_data/gpt_out_ent_attr_200.json','w') as f:
  json.dump(output,f)

### Input - Sentence, quant Output - ent , attr ----- 300 samples

In [43]:
#load the annotated dataset
import json
with open('/content/drive/MyDrive/MTP Project/t5_data/annotated_300.json', 'r') as f:
    samples = json.load(f)
sent={}
for i in samples:
  sent[i['sentence']]=[i['entity'],i['quantity'],i['measuring attribute']]
len(sent)

300

In [44]:
output=[]
for i,v in sent.items():
  p=f"Given the sentence '''{i}''' and the quantity '''{v[1]}''', extract the entity and attribute."

  comp=f"Entity: {v[0]} ; Attribute: {v[2]}"
  line = {'prompt': p, 'completion': comp}

  output.append(line)


In [45]:
output[0]

{'prompt': "Given the sentence '''As of 2016 , Solar cell efficiency exceeds 10 % .''' and the quantity '''10 %''', extract the entity and attribute.",
 'completion': 'Entity: Solar cell ; Attribute: efficiency'}

In [ ]:
len(output)

300

In [ ]:
import json
with open('/content/drive/MyDrive/MTP Project/gpt_data/out_300_ent_attr.jsonl', 'w') as outfile:
  for i in output:
        json.dump(i, outfile)
        outfile.write('\n')


In [ ]:
!openai tools fine_tunes.prepare_data -f '/content/drive/MyDrive/MTP Project/gpt_data/out_300_ent_attr.jsonl'

Analyzing...

- Your file contains 300 prompt-completion pairs
- All prompts end with suffix `''', extract the entity and attribute.`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- All prompts start with prefix `Given the sentence '''`. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- All completions start with prefix `Entity: `. Most of the time you should only add the output data into the completion, without any prefix
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitesp

In [ ]:
!openai api fine_tunes.create -t "/content/drive/MyDrive/MTP Project/gpt_data/out_300_ent_attr_prepared.jsonl" -m davinci

Upload progress: 100% 88.6k/88.6k [00:00<00:00, 94.6Mit/s]
Uploaded file from /content/drive/MyDrive/MTP Project/gpt_data/out_300_ent_attr_prepared.jsonl: file-NHfdG4NxDULYwdc87pDLfqXa
Created fine-tune: ft-OLiUXKgvQfDULdS8KF6nbHT8
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-24 13:34:29] Created fine-tune: ft-OLiUXKgvQfDULdS8KF6nbHT8
[2023-04-24 13:34:58] Fine-tune costs $2.29
[2023-04-24 13:34:58] Fine-tune enqueued. Queue number: 0
[2023-04-24 13:35:00] Fine-tune started



In [ ]:
!openai api fine_tunes.follow -i ft-OLiUXKgvQfDULdS8KF6nbHT8

[2023-04-24 13:34:29] Created fine-tune: ft-OLiUXKgvQfDULdS8KF6nbHT8
[2023-04-24 13:34:58] Fine-tune costs $2.29
[2023-04-24 13:34:58] Fine-tune enqueued. Queue number: 0
[2023-04-24 13:35:00] Fine-tune started
[2023-04-24 13:38:07] Completed epoch 1/4
[2023-04-24 13:39:39] Completed epoch 2/4
[2023-04-24 13:41:11] Completed epoch 3/4
[2023-04-24 13:42:43] Completed epoch 4/4
[2023-04-24 13:43:21] Uploaded model: davinci:ft-personal-2023-04-24-13-43-20
[2023-04-24 13:43:22] Uploaded result file: file-Nmp0z3z7MbjQFkGozu3tVqpM
[2023-04-24 13:43:22] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-04-24-13-43-20 -p <YOUR_PROMPT>


In [ ]:
model_engine='davinci:ft-personal-2023-04-24-13-43-20'
# Set the model_engine variable to the specified engine for fine-tuned model

# Initialize variables
m = 0
out = {}
import time

# Iterate over the test list
for se in test:
    # Create a prompt string using the values from the current element in the test list
    p = f"Given the sentence '''{se['sentence']}''' and the quantity '''{se['quantity']}''', extract the entity and attribute."

    # Generate completions using the fine-tuned model
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=p,
        temperature=0.7,
        max_tokens=30,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        n=1
    )

    # Get the first completion from the generated completions
    message = completions.choices[0].text

    # Print the original sentence and the generated completion
    print(se['sentence'])
    print("*****************************Result**************************")
    print(message.split('\n')[0])
    print('\n\n')

    # Store the generated completion in the out dictionary using the original sentence as the key
    out[se['sentence']] = message.split('\n')[0]

    m = m + 1
    # Delay execution every 20 iterations to avoid rate limits
    if m % 20 == 0:
        time.sleep(65)



In [ ]:
output=[]
# Define regular expressions to match the entity and attribute values
entity_regex = r"Entity:\s*([\w\s]+)"
attribute_regex = r"Attribute:\s*([\w\s]+)"

for i,v in out.items():
  print(i)
  print(f'*******************   RESULT   *******************   ')
  input=v
# Use regular expressions to extract the entity and attribute values from the input string
  entity_match = re.search(entity_regex, input)
  attribute_match = re.search(attribute_regex, input)
# Extract the entity and attribute values from the regular expression matches
  entity_value = entity_match.group(1).strip()
  attribute_value = attribute_match.group(1).strip()
  print(f'Entity----{entity_value}')
  print(f"Attribute ----{attribute_value}")

  print('\n\n')
  d={}
  d['sentence']=i
  d['entity']=entity_value
  d['attribute']=attribute_value
  output.append(d)


In [ ]:
output[10]

{'sentence': 'The bridge carries 60 to 70 percent of commercial truck traffic in the region .',
 'entity': 'bridge',
 'attribute': 'commercial truck traffic share'}

In [ ]:
with open('/content/drive/MyDrive/MTP Project/gpt_data/gpt_out_ent_attr_300.json','w') as f:
  json.dump(output,f)

##input - sen,ent,quant    
##output-attribute -- 200 samples

In [47]:
#load the annotated dataset
import json
with open('/content/drive/MyDrive/MTP Project/t5_data/annotated_200.json', 'r') as f:
    samples = json.load(f)
sent={}
for i in samples:
  sent[i['sentence']]=[i['entity'],i['quantity'],i['attribute']]
len(sent)

200

In [48]:
output=[]
for i,v in sent.items():
  p=f"Given the sentence '''{i}''' and the quantity '''{v[1]}''', along with the entity  '''{v[0]}''', extract the related attribute."

  line = {'prompt': p, 'completion': v[2] }

  output.append(line)


In [49]:
output[0]

{'prompt': "Given the sentence '''Without the spire , Burj Khalifa would be 585 meters tall .''' and the quantity '''585 meters''', along with the entity  '''Burj Khalifa''', extract the related attribute.",
 'completion': 'height'}

In [ ]:
len(output)

200

In [ ]:
import json
with open('/content/drive/MyDrive/MTP Project/gpt_data/out_200_attr.jsonl', 'w') as outfile:
  for i in output:
        json.dump(i, outfile)
        outfile.write('\n')


In [ ]:
!openai tools fine_tunes.prepare_data -f '/content/drive/MyDrive/MTP Project/gpt_data/out_200_attr.jsonl'

Analyzing...

- Your file contains 200 prompt-completion pairs
- All prompts end with suffix `''', extract the related attribute.`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- All prompts start with prefix `Given the sentence '''`. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tunin

In [ ]:
!openai api fine_tunes.create -t "/content/drive/MyDrive/MTP Project/gpt_data/out_200_attr_prepared.jsonl" -m davinci

Upload progress: 100% 60.4k/60.4k [00:00<00:00, 79.8Mit/s]
Uploaded file from /content/drive/MyDrive/MTP Project/gpt_data/out_200_attr_prepared.jsonl: file-2ZtdxU4DG8O0Cjx6dtUPibzu
Created fine-tune: ft-zFrnY1gZ51vP8vKmVoKN5zaA
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-25 17:41:34] Created fine-tune: ft-zFrnY1gZ51vP8vKmVoKN5zaA
[2023-04-25 17:42:30] Fine-tune costs $1.58
[2023-04-25 17:42:31] Fine-tune enqueued. Queue number: 3



In [ ]:
!openai api fine_tunes.follow -i ft-zFrnY1gZ51vP8vKmVoKN5zaA

[2023-04-25 17:41:34] Created fine-tune: ft-zFrnY1gZ51vP8vKmVoKN5zaA
[2023-04-25 17:42:30] Fine-tune costs $1.58
[2023-04-25 17:42:31] Fine-tune enqueued. Queue number: 3
[2023-04-25 17:46:17] Fine-tune is in the queue. Queue number: 2
[2023-04-25 17:46:44] Fine-tune is in the queue. Queue number: 1
[2023-04-25 17:51:13] Fine-tune is in the queue. Queue number: 0
[2023-04-25 17:51:30] Fine-tune started
[2023-04-25 17:53:59] Completed epoch 1/4
[2023-04-25 17:55:00] Completed epoch 2/4
[2023-04-25 17:56:01] Completed epoch 3/4
[2023-04-25 17:57:03] Completed epoch 4/4
[2023-04-25 17:57:48] Uploaded model: davinci:ft-personal-2023-04-25-17-57-48
[2023-04-25 17:57:50] Uploaded result file: file-T6O710Rz3oGgF2HCsQsLD7y6
[2023-04-25 17:57:50] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-04-25-17-57-48 -p <YOUR_PROMPT>


In [ ]:
# Set the model_engine variable to the specified engine for fine-tuned model
model_engine = 'davinci:ft-personal-2023-04-25-17-57-48'

# Initialize variables
out = {}
m = 0
import time

# Iterate over the test list
for se in test:
    # Create a prompt string using the values from the current element in the test list
    p = f"Given the sentence '''{se['sentence']}''' and the quantity '''{se['quantity']}''', along with the entity  '''{se['entity']}''', extract the related attribute."

    # Generate completions using the fine-tuned model
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=p,
        temperature=0.7,
        max_tokens=30,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        n=1
    )

    # Get the first completion from the generated completions
    message = completions.choices[0].text

    # Print the original sentence and the generated completion
    print(se['sentence'])
    print("*****************************Result**************************")
    print(message.split('\n')[0])
    print('\n\n')

    # Store the generated completion in the out dictionary using the original sentence as the key
    out[se['sentence']] = message.split('\n')[0]

    m = m + 1
    # Delay execution every 20 iterations to avoid rate limits
    if m % 20 == 0:
        time.sleep(65)



The total length of this LPG pipeline is 1415 km .
*****************************Result**************************
 total length



Defensive end Rulon Jones led the line with 7 sacks .
*****************************Result**************************
 sacks



'' The diameter of an Oreo cookie is 1.75 inches .
*****************************Result**************************
 diameter



This presence in Russia lasted 3 years .
*****************************Result**************************
 presence



Hyundai owns 33.88 % of Kia .
*****************************Result**************************
 Kia share



The Criollos entered the league 's " Super 6 " semifinals .
*****************************Result**************************
 semifinals participation



Vermont 's share is about 4.5 percent .
*****************************Result**************************
 share



Ballari district has 25 % of India 's Iron ore reserves .
*****************************Result**************************
 Iron ore res

In [ ]:
output = []  # Initialize an empty list to store the output dictionaries

for i, v in out.items():
    d = {}  # Create an empty dictionary

    d['sentence'] = i  # Assign the current sentence to the 'sentence' key in the dictionary

    d['attribute'] = v.strip()  # Assign the attribute value to the 'attribute' key in the dictionary

    output.append(d)  # Append the dictionary to the 'output' list


In [ ]:
output[-1]

{'sentence': 'At the time Ibn Taymiyyah was 42 years old .',
 'attribute': 'age'}

In [ ]:
with open('/content/drive/MyDrive/MTP Project/gpt_data/gpt_out_attr_200.json','w') as f:
  json.dump(output,f)

#input-sen,quan,entity output-attr - 300 samples

In [50]:
#load the annotated dataset
import json
with open('/content/drive/MyDrive/MTP Project/t5_data/annotated_300.json', 'r') as f:
    samples = json.load(f)
sent={}
for i in samples:
  sent[i['sentence']]=[i['entity'],i['quantity'],i['measuring attribute']]
len(sent)

300

In [51]:
output=[]
for i,v in sent.items():
  p=f"Given the sentence '''{i}''' and the quantity '''{v[1]}''', along with the entity  '''{v[0]}''', extract the related attribute."

  line = {'prompt': p, 'completion': v[2] }

  output.append(line)


In [52]:
output[0]

{'prompt': "Given the sentence '''As of 2016 , Solar cell efficiency exceeds 10 % .''' and the quantity '''10 %''', along with the entity  '''Solar cell''', extract the related attribute.",
 'completion': 'efficiency'}

In [ ]:
len(output)

100

In [ ]:
import json
with open('/content/drive/MyDrive/MTP Project/gpt_data/out_300_attr.jsonl', 'w') as outfile:
  for i in output:
        json.dump(i, outfile)
        outfile.write('\n')


In [ ]:
!openai tools fine_tunes.prepare_data -f '/content/drive/MyDrive/MTP Project/gpt_data/out_300_attr.jsonl'

Analyzing...

- Your file contains 100 prompt-completion pairs
- All prompts end with suffix `''', extract the related attribute.`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- All prompts start with prefix `Given the sentence '''`. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tunin

In [ ]:
!openai api fine_tunes.create -t "/content/drive/MyDrive/MTP Project/gpt_data/out_300_attr_prepared.jsonl" -m davinci

Upload progress: 100% 30.0k/30.0k [00:00<00:00, 25.1Mit/s]
Uploaded file from /content/drive/MyDrive/MTP Project/gpt_data/out_300_attr_prepared.jsonl: file-1QwpJB1lis2SUekm6RqMGxvN
Created fine-tune: ft-nSQOu6lV2aQYXHxcdX3DzPNv
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-24 16:05:18] Created fine-tune: ft-nSQOu6lV2aQYXHxcdX3DzPNv
[2023-04-24 16:05:37] Fine-tune costs $0.79
[2023-04-24 16:05:37] Fine-tune enqueued. Queue number: 3



In [ ]:
!openai api fine_tunes.follow -i ft-nSQOu6lV2aQYXHxcdX3DzPNv

[2023-04-24 16:05:18] Created fine-tune: ft-nSQOu6lV2aQYXHxcdX3DzPNv
[2023-04-24 16:05:37] Fine-tune costs $0.79
[2023-04-24 16:05:37] Fine-tune enqueued. Queue number: 3
[2023-04-24 16:07:06] Fine-tune is in the queue. Queue number: 2
[2023-04-24 16:09:49] Fine-tune is in the queue. Queue number: 1
[2023-04-24 16:10:07] Fine-tune is in the queue. Queue number: 0
[2023-04-24 16:11:54] Fine-tune started
[2023-04-24 16:15:02] Completed epoch 1/4
[2023-04-24 16:15:33] Completed epoch 2/4
[2023-04-24 16:16:03] Completed epoch 3/4
[2023-04-24 16:16:34] Completed epoch 4/4
[2023-04-24 16:17:11] Uploaded model: davinci:ft-personal-2023-04-24-16-17-11
[2023-04-24 16:17:12] Uploaded result file: file-NcfM3mAi2lTEyvCrtEelt972
[2023-04-24 16:17:13] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-04-24-16-17-11 -p <YOUR_PROMPT>


In [ ]:
# Set the model_engine variable to the specified engine for fine-tuned model
model_engine='davinci:ft-personal-2023-04-24-16-17-11'

# Initialize variables
out = {}
m = 0
import time

# Iterate over the test list
for se in test:
    # Create a prompt string using the values from the current element in the test list
    p = f"Given the sentence '''{se['sentence']}''' and the quantity '''{se['quantity']}''', along with the entity  '''{se['entity']}''', extract the related attribute."

    # Generate completions using the fine-tuned model
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=p,
        temperature=0.7,
        max_tokens=30,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        n=1
    )

    # Get the first completion from the generated completions
    message = completions.choices[0].text

    # Print the original sentence and the generated completion
    print(se['sentence'])
    print("*****************************Result**************************")
    print(message.split('\n')[0])
    print('\n\n')

    # Store the generated completion in the out dictionary using the original sentence as the key
    out[se['sentence']] = message.split('\n')[0]

    m = m + 1
    # Delay execution every 20 iterations to avoid rate limits
    if m % 20 == 0:
        time.sleep(65)



In [ ]:
output = []  # Initialize an empty list to store the output dictionaries

for i, v in out.items():
    d = {}  # Create an empty dictionary

    d['sentence'] = i  # Assign the current sentence to the 'sentence' key in the dictionary

    d['attribute'] = v.strip()  # Assign the attribute value to the 'attribute' key in the dictionary

    output.append(d)  # Append the dictionary to the 'output' list


In [ ]:
with open('/content/drive/MyDrive/MTP Project/gpt_data/gpt_out_attr_300.json','w') as f:
  json.dump(output,f)

In [ ]:
output[-10]

{'sentence': 'The London demonstration lasted for well over four hours .',
 'attribute': ' duration'}